## **Exploring data with R - visualize data**

There's a wise saying that goes like this: a picture is worth a thousand rows and columns.

Actually, sorry, we made that up - there is no such wise saying 😄. But you get the gist of it, right?

In this notebook, we'll apply basic techniques to analyze data with basic statistics and visualise using graphs with `ggplot2`, a core member of the Tidyverse.

## **Loading our data**

Before we begin, lets load the same data about study hours that we analysed in the previous notebook. We will also recalculate who passed, in the same way as last time. Run the code in the cell below by clicking the **► Run** button to see the data.


In [1]:
library(DBI)
library(RSQLite)
library(tidyverse)

In [2]:
mydb <- dbConnect(RSQLite::SQLite(), "")

In [6]:
ratings <- readr::read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-01-25/ratings.csv')
details <- readr::read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-01-25/details.csv')

ratings %>% 
  slice_head(n = 5)

details %>% 
  slice_head(n = 5)

Rows: 21831 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): name, url, thumbnail
dbl (7): num, id, year, rank, average, bayes_average, users_rated

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 21631 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): primary, description, boardgamecategory, boardgamemechanic, boardg...
dbl (13): num, id, yearpublished, minplayers, maxplayers, playingtime, minpl...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


num,id,name,year,rank,average,bayes_average,users_rated,url,thumbnail
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
105,30549,Pandemic,2008,106,7.59,7.487,108975,/boardgame/30549/pandemic,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXLLjVqA__micro/img/S4tXI3Yo7BtqmBoKINLLVUFsaJ0=/fit-in/64x64/filters:strip_icc()/pic1534148.jpg
189,822,Carcassonne,2000,190,7.42,7.309,108738,/boardgame/822/carcassonne,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOvHfwRA__micro/img/VfLoKzfk3xj26ArmDu55qZ4sysw=/fit-in/64x64/filters:strip_icc()/pic6544250.png
428,13,Catan,1995,429,7.14,6.970,108024,/boardgame/13/catan,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__micro/img/LA4OvGfQ_TXQ-2mhaIFZp2ITWpc=/fit-in/64x64/filters:strip_icc()/pic2419375.jpg
72,68448,7 Wonders,2010,73,7.74,7.634,89982,/boardgame/68448/7-wonders,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__micro/img/9glsOs7zoTbkVpfDt5SHWJm-kRA=/fit-in/64x64/filters:strip_icc()/pic860217.jpg
103,36218,Dominion,2008,104,7.61,7.499,81561,/boardgame/36218/dominion,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HNCODBA__micro/img/PVxqHWOLTb3n-4xe62LJadr_M0I=/fit-in/64x64/filters:strip_icc()/pic394356.jpg


num id    primary    
1 0   30549 Pandemic   
2 1     822 Carcassonne
3 2      13 Catan      
4 3   68448 7 Wonders  
5 4   36218 Dominion   
  description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
1 In Pandemic, several virulent diseases have broken out simultaneously all over the world! The players are disease-fighting specialists whose mission is to treat disease hotspots while researching cures for each of four plagues before they get out of hand.&#10;&#10;The game board depicts several major population centers on Earth. On each turn, a player can use up to four actions to travel between cities, treat infected populaces, discover a cure, or build a research station. A deck of cards provides the players with these abilities, but sprinkled throughout this deck are Epidemic! cards that accelerate and intensify the diseases' activity. A second, separate deck of cards controls the &quot;normal&quot; spread of the infections.&#10;&#10;Taking a unique role within the team, players must plan their strategy to mesh with their specialists' strengths in order 

In [7]:
mydb <- dbConnect(RSQLite::SQLite(), "")
dbWriteTable(mydb, "ratings", ratings)
dbWriteTable(mydb, "details", details)
dbListTables(mydb)

[1] "details" "ratings"

In [8]:
dbGetQuery(mydb, 'SELECT * FROM ratings LIMIT 5')

num,id,name,year,rank,average,bayes_average,users_rated,url,thumbnail
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
105,30549,Pandemic,2008,106,7.59,7.487,108975,/boardgame/30549/pandemic,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXLLjVqA__micro/img/S4tXI3Yo7BtqmBoKINLLVUFsaJ0=/fit-in/64x64/filters:strip_icc()/pic1534148.jpg
189,822,Carcassonne,2000,190,7.42,7.309,108738,/boardgame/822/carcassonne,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOvHfwRA__micro/img/VfLoKzfk3xj26ArmDu55qZ4sysw=/fit-in/64x64/filters:strip_icc()/pic6544250.png
428,13,Catan,1995,429,7.14,6.970,108024,/boardgame/13/catan,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__micro/img/LA4OvGfQ_TXQ-2mhaIFZp2ITWpc=/fit-in/64x64/filters:strip_icc()/pic2419375.jpg
72,68448,7 Wonders,2010,73,7.74,7.634,89982,/boardgame/68448/7-wonders,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__micro/img/9glsOs7zoTbkVpfDt5SHWJm-kRA=/fit-in/64x64/filters:strip_icc()/pic860217.jpg
103,36218,Dominion,2008,104,7.61,7.499,81561,/boardgame/36218/dominion,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HNCODBA__micro/img/PVxqHWOLTb3n-4xe62LJadr_M0I=/fit-in/64x64/filters:strip_icc()/pic394356.jpg


Now, let's get the data ready for further analysis.



In [ ]:
# Read a CSV file into a tibble
df_students <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/grades.csv")

# Remove any rows with missing data
df_students <- df_students %>% 
  drop_na()

# Add a column "Pass" that specifies if a student passed or failed
# Assuming '60' is the grade needed to pass
df_students <- df_students %>% 
  mutate(Pass = Grade >= 60)

# Print the results
df_students


Good job! That went well!

## **Visualizing data with ggplot2**

Data frames provide a great way to explore and analyze rectangular data, but sometimes plotting the data can greatly enhance your ability to analyze the data, see underlying trends and raise new questions.

`Ggplot2` is a package for creating elegant graphics for data analysis in R. Compared with other graphing systems, ggplot2 provides a flexible and intuitive way of creating graphs, by combining independent components of a graphic in a series of iterative steps. This allows you to create visualisations that match your specific needs rather than being limited to sets of predefined graphics.

Now let's see this in action. We'll start with a simple bar chart that shows the grade of each student.


In [ ]:
ggplot(data = df_students) +
  geom_col(mapping = aes(x = Name, y = Grade))


Well, that worked; but the chart could use some improvements to make it clearer what we're looking at. We'll get to that. Let's first walk through the process of creating graphics in ggplot2.

You initialize a graphic using the function `ggplot()` and the data frame to use for the plot. `ggplot(data = df_students)` basically creates an empty graph which you can add layers to using a `+`.

`geom_col()` then adds a layer of bars whose height corresponds to the variables specified by the `mapping` argument. The mapping argument is always paired with `aes()`, which specifies how **variables in the data** are mapped (what goes into aes() are variables found in the data). In our case, we specified, map `Name` to the x axis and `Grade` to the y axis.

And that's it! We'll follow and extend this blueprint to make different types of graphs.

Now, let's improve the visual elements of the plot. For example, the following code:

-   Specifies the color of the bar chart.

-   Adds a title to the chart (so we know what it represents)

-   Adds labels to the X and Y (so we know which axis shows which data)


In [ ]:
# Change the default grey background
theme_set(theme_light())


ggplot(data = df_students) +
  geom_col(mapping = aes(x = Name, y = Grade),
           # Specifiy color and transparency of the bars
           fill = "midnightblue", alpha = 0.7) +
  # Add a title to the chart
  ggtitle("Student Grades") +
  # Add labels to axes
  xlab("Student") +
  ylab("Grade")


Whooa! That's a step in the right direction. We can even improve this further using `ggplot2`'s comprehesive theming system. Themes are a powerful way to customize the non-data components of your plots: i.e. titles, labels, fonts, background, gridlines and legends. You can learn more about modifying components of a theme by running `?theme`.

For instance, let's:

-   Center the title

-   Add a grid (to make it easier to determine the values for the bars)

-   Rotate the X markers (so we can read them)


In [ ]:
ggplot(data = df_students) +
  geom_col(mapping = aes(x = Name, y = Grade),
           fill = "midnightblue", alpha = 0.7) +
  ggtitle("Student Grades") +
  xlab("Student") +
  ylab("Grade") +
  theme(
    # Center the title
    plot.title = element_text(hjust = 0.5),
    
    # Add a grid (to make it easier to determine the bar values
    panel.grid = element_blank(),
    panel.grid.major.y = element_line(color = "gray", linetype = "dashed", size = 0.5),
    
    # Rotate the X markers so we can read them
    axis.text.x = element_text(angle = 90)
    
  )


Good job! Perhaps the bar chart would be more informative if the student names were in a certain order, right? This is a good chance to showcase how to use `dplyr` and `ggplot2` to derive insights from your data.

So, let's reorder the levels of the Name column in descending order, based on the Grade column and then plot this.


In [ ]:
df_students %>% 
  mutate(Name = fct_reorder(Name, Grade, .desc = TRUE)) %>% 
  ggplot() +
  geom_col(mapping = aes(x = Name, y = Grade),
           fill = "midnightblue", alpha = 0.7) +
  ggtitle("Student Grades") +
  xlab("Student") +
  ylab("Grade") +
  theme(
    plot.title = element_text(hjust = 0.5),
    
    panel.grid = element_blank(),
    panel.grid.major.y = element_line(color = "gray", linetype = "dashed", size = 0.5),
    
    axis.text.x = element_text(angle = 90)
    
  )


That's a much better plot - both in aesthetics and information. For instance, we can quickly and easily discern how each student performed.

## Getting started with statistical analysis

Now that you know how to use R to manipulate and visualize data, you can start analyzing it 🎉.

A lot of data science is rooted in *statistics*, so we'll explore some basic statistical techniques.

> **Note**: This is not intended to teach you statistics - that's much too big a topic for this notebook. It will however introduce you to some statistical concepts and techniques that data scientists use as they explore data in preparation for machine learning modeling.

### Descriptive statistics and data distribution

When examining a *variable* (for example a sample of student grades), data scientists are particularly interested in its *distribution* (in other words, how are all the different grade values spread across the sample). The starting point for this exploration is often to visualize the data as a histogram, and see how frequently each value for the variable occurs.

So what `geom` are we going to use? " `geom_histogram`", you'll say - because you are already getting the gist/geom of it 🥳!


In [ ]:
# Visualise distribution of the grades in a histogram
ggplot(data = df_students) +
  geom_histogram(mapping = aes(x = Grade))


Alright. This certainly tells us something about our data - for instance most of the grades seem to be around 50. However, `ggplot2` is extremely flexible and allows us to experiment with different function arguments to better reveal the story behind our data. By looking at `?geom_histogram` we can experiment with arguments such as `binwidth` and `boundary` as shown:



In [ ]:
# Visualise distribution of the grades in a histogram
ggplot(data = df_students) +
  geom_histogram(mapping = aes(x = Grade), , binwidth = 20, boundary = 0.5, fill = "midnightblue", alpha = 0.7) +
  xlab('Grade') +
  ylab('Frequency') +
  theme(plot.title = element_text(hjust = 0.5))


Much better! The histogram for grades is a symmetric shape, where the most frequently occurring grades tend to be in the middle of the range (around 50), with fewer grades at the extreme ends of the scale.

### Measures of central tendency

To understand the distribution better, we can examine so-called *measures of central tendency*; which is a fancy way of describing statistics that represent the "middle" of the data. The goal of this is to try to find a "typical" value. Common ways to define the middle of the data include:

-   The *mean*: A simple average based on adding together all of the values in the sample set, and then dividing the total by the number of samples.

-   The *median*: The value in the middle of the range of all of the sample values.

-   The *mode*: The most commonly occuring value in the sample set^\*^.

Let's calculate these values, along with the minimum and maximum values for comparison, and show them on the histogram.

> Of course, in some sample sets , there may be a tie for the most common value - in which case the dataset is described as *bimodal* or even *multimodal*.

Base *R does not* provide a *function* for finding the *mode*. But worry not, `statip::mfv` returns the most frequent value(s) (or mode(s)) found in a vector. Other pretty awesome workarounds can be found on this [stackoverflow thread](https://stackoverflow.com/questions/2547402/how-to-find-the-statistical-mode).


In [ ]:
# Load statip into the current R sesssion
library(statip)

# Get summary statistics
min_val <- min(df_students$Grade)
max_val <- max(df_students$Grade)
mean_val <- mean(df_students$Grade)
med_val <- median(df_students$Grade)
mod_val <- mfv(df_students$Grade)

# Print the stats
cat(
  "Minimum: ", round(min_val, 2),
   "\nMean: ", round(mean_val, 2),
   "\nMedian: ", round(med_val, 2),
   "\nMode: ", round(mod_val, 2),
   "\nMaximum: ", round(max_val, 2)
)


Now let's incorporate these statistics onto our graph.



In [ ]:
# Plot a histogram
ggplot(data = df_students) +
  geom_histogram(mapping = aes(x = Grade), binwidth = 20, fill = "midnightblue", alpha = 0.7, boundary = 0.5) +
  
# Add lines for the statistics
  geom_vline(xintercept = min_val, color = 'gray33', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mean_val, color = 'cyan', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = 'red', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = mod_val, color = 'yellow', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = max_val, color = 'gray33', linetype = "dashed", size = 1.3 ) +
  
# Add titles and labels
  ggtitle('Data Distribution')+
  xlab('Value')+
  ylab('Frequency')+
  theme(plot.title = element_text(hjust = 0.5))


> `geom_vline()` adds a vertical reference line to a plot.

Good job!

For the grade data, the mean, median, and mode all seem to be more or less in the middle of the minimum and maximum, at around 50.

Another way to visualize the distribution of a variable is to use a *box* plot (sometimes called a *box-and-whiskers* plot). Let's create one for the grade data.


In [ ]:
# Plot a box plot
ggplot(data = df_students) +
  geom_boxplot(mapping = aes(x = 1, y = Grade), fill = "#E69F00", color = "gray23", alpha = 0.7) +
  
  # Add titles and labels
  ggtitle("Data Distribution") +
  xlab("") +
  ylab("Grade") +
  theme(plot.title = element_text(hjust = 0.5))


The box plot shows the distribution of the grade values in a different format to the histogram. The *box* part of the plot shows where the inner two *quartiles* of the data reside - so in this case, half of the grades are between approximately 36 and 63. The *whiskers* extending from the box show the outer two quartiles; so the other half of the grades in this case are between 0 and 36 or 63 and 100. The line in the box indicates the *median* value.

It's often useful to combine histograms and box plots, with the *box plot's orientation changed* to align it with the histogram (in some ways, it can be helpful to think of the histogram as a "front elevation" view of the distribution, and the box plot as a "plan" view of the distribution from above). Since we may need to plot the histograms and box plots for different variables, it will be convenient to write a function. Functions allows you to automate common tasks in a more powerful and general way than copy-and-pasting.

Let's get right to it! Functions in R are generally defined in this fashion:

`name <- function(variables) {return(value)}`

> [patchwork](https://patchwork.data-imaginist.com/) extends `ggplot` API by providing mathematical operators (such as `+` or `/`) for combining multiple plots. Yes, as easy as that!


In [ ]:
library(patchwork)
# Create a function that we can reuse
show_distribution <- function(var_data, binwidth) {
  
  # Get summary statistics by first extracting values from the column
  min_val <- min(pull(var_data))
  max_val <- max(pull(var_data))
  mean_val <- mean(pull(var_data))
  med_val <- median(pull(var_data))
  mod_val <- statip::mfv(pull(var_data))

  # Print the stats
  stats <- glue::glue(
  'Minimum: {format(round(min_val, 2), nsmall = 2)}
   Mean: {format(round(mean_val, 2), nsmall = 2)}
   Median: {format(round(med_val, 2), nsmall = 2)}
   Mode: {format(round(mod_val, 2), nsmall = 2)}
   Maximum: {format(round(max_val, 2), nsmall = 2)}'
  )
  
  # Plot the histogram
  hist_gram <- ggplot(var_data) +
  geom_histogram(aes(x = pull(var_data)), binwidth = binwidth,
                 fill = "midnightblue", alpha = 0.7, boundary = 0.4) +
    
  # Add lines for the statistics
  geom_vline(xintercept = min_val, color = 'gray33', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mean_val, color = 'cyan', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = 'red', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = mod_val, color = 'yellow', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = max_val, color = 'gray33', linetype = "dashed", size = 1.3 ) +
    
  # Add titles and labels
  ggtitle('Data Distribution') +
  xlab('')+
  ylab('Frequency') +
  theme(plot.title = element_text(hjust = 0.5))
  
  # Plot the box plot
  bx_plt <- ggplot(data = var_data) +
  geom_boxplot(mapping = aes(x = pull(var_data), y = 1),
               fill = "#E69F00", color = "gray23", alpha = 0.7) +
    
    # Add titles and labels
  xlab("Value") +
  ylab("") +
  theme(plot.title = element_text(hjust = 0.5))
  
  
  # To return multiple outputs, use a `list`
  return(
    
    list(stats,
         # Combine histogram and box plot using library patchwork
         hist_gram / bx_plt)
    
        ) # End of returned outputs
  
} # End of function


Now that the `show_distribution()` function is done for, let's get a variable/column to examine and then call the function.



In [ ]:
# Get the variable to examine
col <- df_students %>% 
  select(Grade)

# Call the function
show_distribution(var_data = col, binwidth = 20)


All of the measurements of central tendency are right in the middle of the data distribution, which is symmetric with values becoming progressively lower in both directions from the middle.

To explore this distribution in more detail, you need to understand that statistics is fundamentally about taking *samples* of data and using probability functions to *extrapolate information* about the full *population* of data. For example, the student data consists of 22 samples, and for each sample there is a grade value. You can think of each sample grade as a variable that's been randomly selected from the set of all grades awarded for this course. With enough of these random variables, you can calculate something called a *probability density function*, which estimates the distribution of grades for the full population.

A density plot is a representation of the distribution of a numeric variable. It is a smoothed version of the histogram and is often used in the same kind of situation.

> `geom_density()` computes and draws a kernel density estimate, which is a smoothed version of the histogram.


In [ ]:
# Create a function that returns a density plot
show_density <- function(var_data) {
  
  # Get statistics
  mean_val <- mean(pull(var_data))
  med_val <- median(pull(var_data))
  mod_val <- statip::mfv(pull(var_data))
  
  
  # Plot the density plot
  density_plot <- ggplot(data = var_data) +
  geom_density(aes(x = pull(var_data)), fill="orangered", color="white", alpha=0.4) +
    
  # Add lines for the statistics
  geom_vline(xintercept = mean_val, color = 'cyan', linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = 'red', linetype = "dashed", size = 1.3 ) +
  geom_vline(xintercept = mod_val, color = 'yellow', linetype = "dashed", size = 1.3 ) +
    
  # Add titles and labels
  ggtitle('Data Density') +
  xlab('') +
  ylab('Density') +
  theme(plot.title = element_text(hjust = 0.5))
  
  
  
  return(density_plot) # End of returned outputs
  
} # End of function


# Get the density of Grade
col <- df_students %>% select(Grade)
show_density(var_data = col)


As expected from the histogram of the sample, the density shows the characteristic "bell curve" of what statisticians call a *normal* distribution with the mean and mode at the center and symmetric tails.

## **Summary**

Well done! There were a number of new concepts in here, so let's summarise.

Here we have:

1.  Made graphs with ggplot2

2.  Seen how to customise these graphs

3.  Calculated basic statistics, such as medians

4.  Looked at the spread of data using box plots and histograms

5.  Learned about samples vs populations

6.  Estimated what the population of graphs might look like from a sample of grades.

In our next notebook we will look at spotting unusual data, and finding relationships between data.

## **Further Reading**

To learn more about the R packages and concepts you explored in this notebook, see the following documentation:

-   [Tidyverse packages](https://www.tidyverse.org/packages/)

-   [Patchwork](https://patchwork.data-imaginist.com/)

-   [Functions with R](https://skirmer.github.io/presentations/functions_with_r.html#1)
